In [1]:
import utils
from utils import *
from glob import glob
import pandas as pd

In [2]:
# set paths
msf_filenames = glob('data/msfragger/*')
ap_filenames = glob('data/alphapept/*')
mq_filenames = glob('data/maxquant/*')
mm_filenames = sorted(glob('data/metamorpheus/*.tsv'))
mm_filenames_all = sorted(glob('data/metamorpheus/*.psmtsv'))

In [3]:
# make dataframes and add sample column by using read_files function
msf_dfs = read_files(msf_filenames)
mq_dfs = read_files(mq_filenames)
mm_dfs = read_files(mm_filenames)
mm_dfs_all = read_files(mm_filenames_all)
ap_dfs = [pd.read_csv(filename).assign(sample=filename.split('\\')[-1].split('.')[0]) for filename in ap_filenames]

In [ ]:
# manually add fixed modification on C in MaxQuant data
for df in mq_dfs:
    df['Modified sequence'] = df['Modified sequence'].str.replace('C', 'C(Carbamidomethylation (C))')

In [4]:
# parse & merge data
msf = pd.concat([MSF_parser(df) for df in msf_dfs])
ap = pd.concat([AP_parser(df) for df in ap_dfs])
mq = pd.concat([MQ_parser(df) for df in mq_dfs])
mm = pd.concat([MM_parser(df, df_all) for df, df_all in zip(mm_dfs, mm_dfs_all)])

In [5]:
# merge different software outputs
merge_cols = ['sample', 'peptide', 'proteinID', 'mods', 'mods_type']
merged_df = msf.merge(ap, how='outer', left_on=merge_cols, right_on=merge_cols).merge(mq, how='outer', left_on=merge_cols, right_on=merge_cols).merge(mm, how='outer', left_on=merge_cols, right_on=merge_cols)

In [7]:
# save files
merged_df.to_csv('results/mega_merged_data.csv', index=False)

# for sample, sub_df in merged_df.groupby('sample'):
#     sub_df.to_csv(f'results/{sample}_merged_data.csv', index=False)